In [30]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter
import pickle
import os
from datetime import datetime
import tensorflow as tf

In [31]:
label_dic = {
    'bicycle' : 0,
    'horse' : 1,
    'ship' : 2,
    'truck' : 3
}

In [32]:
image_set = {}
resizing = (32, 32)

for label in label_dic:
    image_dir = os.getcwd() + '/' + label
    labeled_image = []

    os.walk(image_dir)

In [33]:
image_set = {}
resizing = (32, 32)

for label in label_dic:
    image_dir = os.getcwd() + '/' + label
    labeled_image = []

    for path, dir, files in os.walk(image_dir):
        for file in files:
            image_dir = path + '/' + file
            img = Image.open(image_dir)
            img = img.resize(resizing)
            if not img.format == "RGB": # 이미지의 포맷이 RGB가 아닐 경우, RGB로 convert 시킴
                img = img.convert("RGB")
            labeled_image.append(np.array(img))
    
    image_set[label] = np.array(labeled_image)

In [34]:
train_class_size = 200
test_class_size = 20


data_set = {
    'train_image': np.empty(1),
    'train_label': np.empty(1),
    'test_image': np.empty(1),
    'test_label': np.empty(1),
}

In [35]:
train_class_size = 200
test_class_size = 20

data_set = {
    'train_image' : np.empty((1,resizing[0],resizing[1],3)),
    'train_label' : np.empty(1),
    'test_image' : np.empty((1,resizing[0],resizing[1],3)),
    'test_label' : np.empty(1)
}

for label in image_set:
    label_index = np.random.randint(len(image_set[label]), size=train_class_size + test_class_size)
    
    train_image = image_set[label][label_index[:train_class_size]]
    test_image = image_set[label][label_index[train_class_size:train_class_size+test_class_size]]
    train_label = np.repeat(label_dic[label], train_class_size)
    test_label = np.repeat(label_dic[label], test_class_size)
    
    data_set['train_image'] = np.concatenate((data_set['train_image'], train_image), axis = 0)
    data_set['test_image'] = np.concatenate((data_set['test_image'], test_image), axis = 0)
    data_set['train_label'] = np.concatenate((data_set['train_label'], train_label), axis = 0)
    data_set['test_label'] = np.concatenate((data_set['test_label'], test_label), axis = 0) # np.concatenate는 cbind, rbind 같은 기능

data_set['train_image'] = data_set['train_image'][1:]
data_set['train_label'] = data_set['train_label'][1:]
data_set['test_image'] = data_set['test_image'][1:]
data_set['test_label'] = data_set['test_label'][1:] # 처음에 넣어줬던 무의미한 값을 빼주는 단계 

In [36]:
data_set['train_image'].shape, data_set['test_image'].shape

((800, 32, 32, 3), (80, 32, 32, 3))

In [37]:
data_set['train_label'].shape, data_set['test_label'].shape

((800,), (80,))

### VGG NET

In [56]:
tf.reset_default_graph()

In [57]:
x = tf.placeholder(dtype=tf.float32, shape=[None,32, 32,3])
y = tf.placeholder(dtype=tf.int32, shape=[None,1])
y_one_hot = tf.one_hot(y, 4)
y_one_hot = tf.reshape(y_one_hot, [-1, 4])

In [58]:
w1 = tf.Variable(tf.random_normal(shape=[3,3,3,64], stddev=0.01))
L1 = tf.nn.conv2d(filter=w1, input=x, strides=[1,1,1,1], padding="SAME")
L1 = tf.nn.relu(L1)
w1_1 = tf.Variable(tf.random_normal(shape=[3,3,64,64], stddev=0.01))
L1_1 = tf.nn.conv2d(filter=w1_1, input=L1, strides=[1,1,1,1], padding="SAME")
L1_1 = tf.nn.relu(L1_1)

L1_1 = tf.nn.max_pool(L1_1, strides=[1,2,2,1], ksize=[1,2,2,1], padding="SAME") # ksize가모지 ? 

In [59]:
L1_1

<tf.Tensor 'MaxPool:0' shape=(?, 16, 16, 64) dtype=float32>

In [60]:
w2 = tf.Variable(tf.random_normal(shape=[3,3,64,128], stddev=0.01))
L2 = tf.nn.conv2d(filter=w2, input=L1_1, strides=[1,1,1,1], padding="SAME")
L2 = tf.nn.relu(L2)

w2_1 = tf.Variable(tf.random_normal(shape=[3,3,128,128], stddev=0.01))
L2_2 = tf.nn.conv2d(filter=w2_1, input=L2, strides=[1,1,1,1], padding="SAME")
L2_2 = tf.nn.relu(L2_2)
L2_2 = tf.nn.max_pool(L2_2, padding="SAME", strides=[1,2,2,1], ksize=[1,2,2,1])  

In [61]:
w3 = tf.Variable(tf.random_normal(shape=[3,3,128,256], stddev=0.01))
L3 = tf.nn.conv2d(filter=w3, input=L2_2, strides=[1,1,1,1], padding="SAME")
L3 = tf.nn.relu(L3)
w4 = tf.Variable(tf.random_normal(shape=[3,3,256,256], stddev=0.01))
L4 = tf.nn.conv2d(filter=w4, input=L3, strides=[1,1,1,1], padding="SAME")
L4 = tf.nn.relu(L4)
w5 = tf.Variable(tf.random_normal(shape=[3,3,256,256], stddev=0.01))
L5 = tf.nn.conv2d(filter=w5, input=L4, strides=[1,1,1,1], padding="SAME")
L5 = tf.nn.relu(L5)

L5 = tf.nn.max_pool(L5, padding="SAME", strides=[1,2,2,1], ksize=[1,2,2,1])  

In [62]:
L5

<tf.Tensor 'MaxPool_2:0' shape=(?, 4, 4, 256) dtype=float32>

In [63]:
w6 = tf.Variable(tf.random_normal(shape=[3,3,256,512], stddev=0.01))
L6 = tf.nn.conv2d(filter=w6, input=L5, strides=[1,1,1,1], padding="SAME")
L6 = tf.nn.relu(L6)
w7 = tf.Variable(tf.random_normal(shape=[3,3,512,512], stddev=0.01))
L7 = tf.nn.conv2d(filter=w7, input=L6, strides=[1,1,1,1], padding="SAME")
L7 = tf.nn.relu(L7)

L7 = tf.nn.max_pool(L7, padding="SAME", strides=[1,2,2,1], ksize=[1,2,2,1])  

In [64]:
L7

<tf.Tensor 'MaxPool_3:0' shape=(?, 2, 2, 512) dtype=float32>

In [65]:
L7_flat = tf.reshape(L7, shape=[-1,2*2*512]) 
w8 = tf.Variable(tf.random_normal(shape=[2*2*512, 100], stddev=0.01))
b1 = tf.Variable(tf.random_normal(shape = [100]))
FC1 = tf.matmul(L7_flat, w8) +b1

w9 = tf.Variable(tf.random_normal(shape=[100, 100], stddev=0.01))
b2 = tf.Variable(tf.random_normal(shape = [100]))
FC2 = tf.matmul(FC1, w9) +b2

w10 = tf.Variable(tf.random_normal(shape=[100, 4], stddev=0.01))
b3 = tf.Variable(tf.random_normal(shape = [4]))
logits = tf.matmul(FC2, w10) +b3

In [66]:
n_epochs = 10
batch_size = 20
lr = 0.01

In [67]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_one_hot))
optimizer = tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.9)
training_op = optimizer.minimize(loss)
correct = tf.equal(tf.argmax(logits,1), tf.argmax(y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
init = tf.global_variables_initializer()

In [68]:
train_y = np.reshape(data_set['train_label'], newshape=[-1,1])
test_y = np.reshape(data_set['test_label'], newshape=[-1,1])

In [69]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(data_set['train_image'].shape[0]//batch_size):
            index = np.random.randint(train_class_size*4 ,size = batch_size)
            sess.run(training_op, feed_dict={x : data_set['train_image'][index], y : train_y[index]})
            
        acc_train = accuracy.eval(feed_dict={x : data_set['train_image'][index], y : train_y[index]})
        acc_test = accuracy.eval(feed_dict={x : data_set['test_image'], y : test_y})
        print("Epoch:", epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

Epoch: 0 Train accuracy: 0.3 Test accuracy: 0.25
Epoch: 1 Train accuracy: 0.2 Test accuracy: 0.25
Epoch: 2 Train accuracy: 0.4 Test accuracy: 0.25
Epoch: 3 Train accuracy: 0.25 Test accuracy: 0.25
Epoch: 4 Train accuracy: 0.25 Test accuracy: 0.25
Epoch: 5 Train accuracy: 0.15 Test accuracy: 0.25
Epoch: 6 Train accuracy: 0.35 Test accuracy: 0.25
Epoch: 7 Train accuracy: 0.3 Test accuracy: 0.25
Epoch: 8 Train accuracy: 0.25 Test accuracy: 0.25
Epoch: 9 Train accuracy: 0.25 Test accuracy: 0.25
